In [0]:
!pip install catboost==0.20.2

     |████████████████████████████████| 63.9MB 57kB/s 


In [0]:
import pandas as pd
import numpy as np
import xgboost as xgb
import catboost as cb
import lightgbm as lgbm
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math
import scipy.stats as st
%matplotlib inline
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)


In [0]:
df1 = pd.read_csv("/content/drive/My Drive/PDDA/Data3_1_2020/train.csv")
df2 = pd.read_csv("/content/drive/My Drive/PDDA/Data3_1_2020/test.csv")
valid = pd.read_csv("/content/drive/My Drive/PDDA/Part_Of_Expected_Results/real_result_20perc.csv")

In [0]:
# remove all rows that contains missing value
df1.replace(['-999', -999], np.nan, inplace=True)


df2.replace(['-999', -999], np.nan, inplace=True)


# Dropping Nan Values

In [0]:
df = df1.dropna()

# Feature Engineering

In [0]:
train = df.drop(['DTC'],axis=1)
train['CNC'] = np.where((train.CNC >= 0.7),0.7,train.CNC)
train['HRM'] = np.where((train.HRM >= 10),10,train.HRM)
train['HRD'] = np.where((train.HRD >= 10),10,train.HRD)
train['GR'] = np.where((train.GR >= 150),150,train.GR)

# Same Feature Engr on Valdation set

In [0]:
valid1 =valid

valid1['CNC'] = np.where((valid1.CNC >= 0.7),0.7,valid1.CNC)
valid1['HRM'] = np.where((valid1.HRM >= 10),10,valid1.HRM)
valid1['HRD'] = np.where((valid1.HRD >= 10),10,valid1.HRD)
valid1['GR'] = np.where((valid1.GR >= 150),150,valid1.GR)


In [0]:
train_x = train.drop('DTS',axis=1)
train_y = train['DTS']
params = {"max_depth": [5,10, 15],
         "learning_rate": [0.01, 0.05, 0.1,],
         "n_estimators": [100, 250,300,500,1000]}
params_cb = {'depth': [5, 10, 15],
            'learning_rate': [0.01, 0.05, 0.1],
            'iterations': [100, 250]}
test_y = valid['DTS']
test_x = valid1.drop(['DTC', 'DTS'], axis=1)
n_splits = 5
max_train_size = len(train_x) // (n_splits+1)

In [0]:
%%time
model_lg = lgbm.LGBMRegressor()
grid_search_lg = GridSearchCV(model_lg, param_grid=params, cv=TimeSeriesSplit(n_splits=n_splits, max_train_size=max_train_size), verbose=1, n_jobs=-1)
grid_search_lg.fit(train_x, train_y)

Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   19.5s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  1.4min


CPU times: user 1.86 s, sys: 120 ms, total: 1.98 s
Wall time: 1min 43s


[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:  1.7min finished


In [0]:
predict_DTS_lg1 = grid_search_lg.predict(test_x)
print('LightGBM RMSE Score: {}'.format(math.sqrt(mean_squared_error(test_y, predict_DTS_lg1))))
print("BEST PARAMETERS: " + str(grid_search_lg.best_params_))

LightGBM RMSE Score: 23.504229064844107
BEST PARAMETERS: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}


# Making Xgboost DTS prediction

In [0]:
train = df1.drop(['DTC'],axis=1)
train1 = df1.drop(['DTC'],axis=1)
train1['CAL'] = train['CAL'].fillna(train['CAL'].median())
train1['CNC'] = train['CNC'].fillna(train['CNC'].median())
train1['GR'] = train['GR'].fillna(train['GR'].median())
train1['HRD'] = train['HRD'].fillna(train['HRD'].median())
train1['HRM'] = train['HRM'].fillna(train['HRM'].median())
train1['PE'] = train['PE'].fillna(train['PE'].median())
train1['ZDEN'] = train['ZDEN'].fillna(train['ZDEN'].median())

In [0]:
train1 = train1.dropna()
train = train1
train.describe()

,CAL,CNC,GR,HRD,HRM,PE,ZDEN,DTS
count,25278.000000,25278.000000,25278.000000,25278.000000,25278.000000,25278.000000,25278.000000,25278.000000
mean,8.516952,0.259706,50.032116,19.727229,16.824116,4.279824,2.417345,180.655730
std,1.785502,2.759995,54.917419,378.672530,483.181868,4.184133,0.172678,81.141960
min,5.930400,0.014500,-0.146000,0.054100,0.061600,-0.023200,0.680600,80.580400
25%,6.891925,0.124700,18.329275,0.887925,0.883100,0.051900,2.255000,129.446625
50%,8.578100,0.186700,37.082200,1.854100,1.862100,4.591900,2.461000,144.593050
75%,8.707575,0.299275,60.753975,3.366050,3.488275,7.095625,2.559400,191.475125
max,21.064200,365.885000,1470.253400,10000.000000,60467.761700,28.106400,3.259700,487.438400


In [0]:
train['CNC'] = np.where((train.CNC >= 0.7),0.7,train.CNC)
train['HRM'] = np.where((train.HRM >= 10),10,train.HRM)
train['HRD'] = np.where((train.HRD >= 10),10,train.HRD)
train['GR'] = np.where((train.GR >= 200),200,train.GR)

In [0]:
valid1 =valid
valid1['CNC'] = np.where((valid1.CNC >= 0.7),0.7,valid1.CNC)
valid1['HRM'] = np.where((valid1.HRM >= 10),10,valid1.HRM)
valid1['HRD'] = np.where((valid1.HRD >= 10),10,valid1.HRD)
valid1['GR'] = np.where((valid1.GR >= 200),200,valid1.GR)

In [0]:
valid1.describe()

,CAL,CNC,GR,HRD,HRM,PE,ZDEN,DTC,DTS
count,2217.000000,2217.000000,2217.000000,2217.000000,2217.000000,2217.000000,2217.000000,2217.000000,2217.000000
mean,8.634826,0.158620,27.902927,3.427507,3.773550,7.360546,2.476402,76.716787,145.373654
std,0.044438,0.090789,28.096775,2.498131,2.672341,1.255721,0.150628,14.257791,44.345398
min,8.500000,0.009800,1.235100,0.086100,0.154800,4.780900,2.033400,53.164700,86.306500
25%,8.625000,0.092000,8.535300,1.770200,1.857600,6.526200,2.377200,66.169800,119.515300
50%,8.625000,0.128900,19.375200,2.772100,3.197000,7.881300,2.535500,71.417900,130.290800
75%,8.671900,0.214200,37.201300,4.511500,5.029100,8.308300,2.582500,85.807000,146.235700
max,8.781300,0.543400,150.000000,10.000000,10.000000,13.741000,3.006900,120.351200,342.425400


In [0]:
X = train.drop('DTS',axis=1)
y = train['DTS']
test = valid1.drop(['DTC','DTS'],axis=1)

In [0]:
best_xgb_model = xgb.XGBRegressor(colsample_bytree=0.8606,
                 gamma=4.794,                 
                 learning_rate=0.05,
                 max_depth=7,
                 min_child_weight=14.03,
                 n_estimators=1000,                                                                    
                 subsample=0.9863,
                 seed=1234)


In [0]:
best_xgb_model.fit(X,y)

[07:48:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8606, gamma=4.794,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=7, min_child_weight=14.03, missing=None,
             n_estimators=1000, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=1234, silent=None, subsample=0.9863, verbosity=1)

In [0]:
xgb1_DTS_preds = best_xgb_model.predict(test)
real_DTS_values = valid['DTS']
print('Root Mean Square Error is:', '{:.5f}'.format(np.sqrt(mean_squared_error(real_DTS_values, xgb1_DTS_preds))))

Root Mean Square Error is: 24.23341


# Making Catboost DTS Prediction

In [0]:
train = df.drop(['DTC'],axis=1)
train['CNC'] = np.where((train.CNC >= 0.7),0.7,train.CNC)
train['HRM'] = np.where((train.HRM >= 10),10,train.HRM)
train['HRD'] = np.where((train.HRD >= 10),10,train.HRD)
train['GR'] = np.where((train.GR >= 200),200,train.GR)

valid1 =valid

valid1['CNC'] = np.where((valid1.CNC >= 0.7),0.7,valid1.CNC)
valid1['HRM'] = np.where((valid1.HRM >= 10),10,valid1.HRM)
valid1['HRD'] = np.where((valid1.HRD >= 10),10,valid1.HRD)
valid1['GR'] = np.where((valid1.GR >= 200),200,valid1.GR)


In [0]:
X=train.drop(["DTS"],axis =1)
y = train.DTS

In [0]:
test = valid1.drop(['DTC','DTS'],axis=1)

In [0]:
from catboost import CatBoostRegressor
errcb2=[]
y_pred_totcb2=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
fold=KFold(n_splits=20)#15#5#10
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m2  = CatBoostRegressor( n_estimators=500,eval_metric='RMSE',learning_rate=0.2, random_seed= 1234, use_best_model=True,)
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=200,verbose=100,)#erly100
    preds=m2.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    errcb2.append(np.sqrt(mean_squared_error(y_test,preds)))
    p2 = m2.predict(test)
    y_pred_totcb2.append(p2)
np.mean(errcb2)

0:	learn: 67.2225051	test: 67.2225051	test1: 86.3207022	best: 86.3207022 (0)	total: 6.81ms	remaining: 3.4s
100:	learn: 8.6392078	test: 8.6392078	test1: 26.7406785	best: 22.6952782 (13)	total: 637ms	remaining: 2.52s
200:	learn: 7.3175255	test: 7.3175255	test1: 26.6606314	best: 22.6952782 (13)	total: 1.22s	remaining: 1.81s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 22.69527817
bestIteration = 13

Shrink model to first 14 iterations.
err:  22.69527813550876
0:	learn: 64.3721083	test: 64.3721083	test1: 133.7035599	best: 133.7035599 (0)	total: 8.08ms	remaining: 4.03s
100:	learn: 8.7615146	test: 8.7615146	test1: 17.6470929	best: 17.1418178 (26)	total: 588ms	remaining: 2.32s
200:	learn: 7.3713999	test: 7.3713999	test1: 16.9594592	best: 16.9448831 (189)	total: 1.19s	remaining: 1.77s
300:	learn: 6.6601311	test: 6.6601311	test1: 17.2053199	best: 16.9448831 (189)	total: 1.76s	remaining: 1.16s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 16.9448831
bes

12.804399934366774

In [0]:
cat_DTS_preds = np.mean(y_pred_totcb2, 0)

In [0]:
real_DTS_values = valid['DTS']
print('Root Mean Square Error is:', '{:.5f}'.format(np.sqrt(mean_squared_error(real_DTS_values, cat_DTS_preds))))

Root Mean Square Error is: 25.92826


# Checking Performance of Stacking Best 3 models

In [0]:
results['final'] = results['xgb_pred']*0.5 + results['gbr_pred']*0.50

In [0]:
stack1_results = (predict_DTS_lg1 + xgb1_DTS_preds + cat_DTS_preds)

In [0]:
stack1 = (0.8*predict_DTS_lg1) + 0.2*(0.5*xgb1_DTS_preds + 0.5*cat_DTS_preds)

In [0]:
print(cat_DTS_preds)
print(predict_DTS_lg1)
print(xgb1_DTS_preds)

[130.63793234 173.09611358 179.48704373 ... 171.82759232 119.05598592
 135.13702046]
[125.11330298 172.53376475 183.36965868 ... 168.41358328 113.63495963
 131.5982824 ]
[123.32403 169.45142 177.19157 ... 168.19225 116.1639  131.64407]


In [0]:
print(stack1)

[126.04714158 171.90376477 180.8544838  ... 169.21175109 115.62245186
 132.49441469]


In [0]:
a = np.array([[1, 2], [3, 4]])

In [0]:
pred_array = pd.DataFrame()

In [0]:
pred_array['LGBM'] = predict_DTS_lg1
pred_array['XGB'] = xgb1_DTS_preds
pred_array['CAT'] = cat_DTS_preds

In [0]:
pred_array['MEAN'] = np.mean(pred_array,axis=1)

In [0]:
pred_array['MEAN'].head()

0    126.358421
1    171.693765
2    180.016092
3    147.417469
4    143.235705
Name: MEAN, dtype: float64

In [0]:
stack2 = pred_array['MEAN']

# Checking Performance of Both stacks

In [0]:
print('Root Mean Square Error is:', '{:.5f}'.format(np.sqrt(mean_squared_error(real_DTS_values, stack1))))

Root Mean Square Error is: 23.65873


In [0]:
print('Root Mean Square Error is:', '{:.5f}'.format(np.sqrt(mean_squared_error(real_DTS_values, stack2))))

Root Mean Square Error is: 24.22970


# Building other Regressors

### Random Forest Regressor

In [0]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [0]:
train = df.drop(['DTC'],axis=1)
train['CNC'] = np.where((train.CNC >= 0.7),0.7,train.CNC)
train['HRM'] = np.where((train.HRM >= 10),10,train.HRM)
train['HRD'] = np.where((train.HRD >= 10),10,train.HRD)
train['GR'] = np.where((train.GR >= 150),150,train.GR)

In [0]:
valid1 =valid

valid1['CNC'] = np.where((valid1.CNC >= 0.7),0.7,valid1.CNC)
valid1['HRM'] = np.where((valid1.HRM >= 10),10,valid1.HRM)
valid1['HRD'] = np.where((valid1.HRD >= 10),10,valid1.HRD)
valid1['GR'] = np.where((valid1.GR >= 150),150,valid1.GR)


In [0]:
X = train.drop('DTS',axis=1)
y = train['DTS']

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_squared_error
from sklearn.model_selection import GridSearchCV
# train test split
# please remember to use random_state for all randomization steps, this will ensure we get the same results 
# as yours during the validation phase. 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1234)


In [0]:
def grid_search(clf, param_grid, X_train=X_train, y_train=y_train):
    """
    Fits a classifier to its training data and prints its ROC AUC score.
    
    INPUT:
    - clf (classifier): classifier to fit
    - param_grid (dict): classifier parameters used with GridSearchCV
    - X_train (DataFrame): training input
    - y_train (DataFrame): training output
            
    OUTPUT:
    - classifier: input classifier fitted to the training data
    """
    # cv uses StratifiedKFold
    # scoring r2 as parameter
    grid = GridSearchCV(estimator=clf, 
                        param_grid=param_grid, 
                        scoring='r2', 
                        cv=5)
    grid.fit(X_train, y_train)
    print(grid.best_score_)
    
    return grid.best_estimator_

In [0]:
# Random forest model
RF = RandomForestRegressor(n_estimators=1000, random_state=1234)
RF_best = grid_search(RF, {})

In [0]:
RF_DTS_preds = RF_best.predict(test)
real_DTS_values = valid['DTS']
print('Root Mean Square Error is:', '{:.5f}'.format(np.sqrt(mean_squared_error(real_DTS_values, RF_DTS_preds))))